In [1]:
import re
import time
import json
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Вспомогательные функции
def check_for_none(x):
    if x is None:
        return ''
    else:
        return x.text

In [3]:
# Вспомогательные функции для Скрапинга
def get_random_ua():
    random_ua = ''
    ua_file = 'ua_file.txt'
    try:
        with open(ua_file) as f:
            lines = f.readlines()
        if len(lines) > 0:
            prng = np.random.RandomState()
            index = prng.permutation(len(lines) - 1)
            idx = np.asarray(index, dtype=np.integer)[0]
            random_proxy = lines[int(idx)]
    except Exception as ex:
        print('Exception in random_ua')
        print(str(ex))
    finally:
        return random_ua

# Parsing

## Avito

In [4]:
main_url = 'https://www.avito.ru'
delays = [7, 4, 6, 10, 5, 8]

In [ ]:
for i in range(1,101):
    add_url = '/rossiya/avtomobili?p='+str(i)
    
    final_data = []
    pred_ap = {}
    if i%10 == 0:
        print(f'Страница {i} успешно обработана')
    # Короткий текст объявления в шаблоне поиска
    url = main_url + add_url
    user_agent = get_random_ua()
    headers = { 'user-agent': user_agent}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text)
    ap = soup.findAll('div', {'class': 'snippet-horizontal'})
    for a in ap:  
        delay = np.random.choice(delays)
        time.sleep(delay)
        pred_ap['main_title'] = check_for_none(a.find('div', {'class': 'snippet-title-row'}))
        pred_ap['price_comment'] = check_for_none(a.find('span', {'class': 'snippet-tag'}))
        pred_ap['price'] = check_for_none(a.find('span', {'class': 'snippet-price'}))
        pred_ap['specific-params'] = check_for_none(a.find('div', {'class': 'specific-params_block'}))
        pred_ap['address'] = check_for_none(a.find('div', {'class': 'item-address'}))

        ########################################################################
        # Полный текст внутри объявления
        full_app_dict = {}

        url = main_url + a.find('a', {'class': 'snippet-link'}, href=True)['href']
        
        user_agent = get_random_ua()
        headers = { 'user-agent': user_agent}
        r = requests.get(url, headers=headers)
        
        soup = BeautifulSoup(r.text)
        
        with open("logs/log.txt", "a") as myfile:
            myfile.write(url+'\n')
        
        fap = soup.find('div', {'class': 'item-params'})
        if fap is not None:           
            li = fap.findAll('li', {'class':'item-params-list-item'})
            for l in li:
                full_app_dict[l.text.split(':')[0].strip()] = l.text.split(':')[1].strip()
            full_app_dict['address'] = check_for_none(soup.find('div', {'class':'item-address'}))
            full_app_dict['full_text_app'] = check_for_none(soup.find('div', class_=re.compile('item-description')))

            ########################################################################
            # Дополенная информация об объекте
            adv_info = {}

            fap_adv = soup.find('ul', class_=re.compile('advanced-params'))
            if fap_adv is not None:
                li = fap_adv.findAll('li', {'class': 'advanced-params-param'})
                for l in li:
                    param_list = []
                    title = l.find('div', {'class': 'advanced-params-param-title'}).text
                    item_list = l.find('ul', {'class': 'advanced-params-param-list'})  
                    for il in item_list.findAll('li', {'class': 'advanced-params-param-item'}):
                            param_list.append(il.text)    
                    adv_info[title] = param_list

            ########################################################################
            #Дополнительная информация по цене в объявлении  
            add_price = {}

            js = soup.find('div', class_ = re.compile('js-ssr'))
            if js is not None:    
                add_info = json.loads(js['data-props'])
                add_price['title'] = add_info['title']
                add_price['subtitle'] = add_info['subtitle']
                add_price['priceType'] = add_info['priceType']
                add_price['marketPrice'] = add_info['marketPrice']
                add_price['isVisibleForUser'] = add_info['isVisibleForUser']
                add_price['price'] = add_info['price']
            else:
                add_price['title'] = ''
                add_price['subtitle'] = ''
                add_price['priceType'] = ''
                add_price['marketPrice'] = ''
                add_price['isVisibleForUser'] = ''
                add_price['price'] = ''

            ########################################################################
            # Технические характеритики модели
            spec_table = {}
            spec_link = soup.find('a', {'class': 'item-view-specification-link'}, href=True)
            if spec_link is not None:
                url = main_url + spec_link['href']
                user_agent = get_random_ua()
                headers = { 'user-agent': user_agent}
                r = requests.get(url,headers=headers)
                soup = BeautifulSoup(r.text)
                table = soup.findAll('table', class_=re.compile('specification-table'))
                for tab in table:
                    tr = tab.findAll('tr')
                    for t in tr:
                        td = t.findAll('td')
                        spec_table[td[0].text] = td[1].text

            # Собираем все данныe в один DataFrame
            final_data.append({**spec_table, **add_price, **pred_ap, **adv_info, **full_app_dict})
        else:
            continue
    pd.DataFrame(final_data).to_excel('data/avito/page'+str(i)+'.xlsx')

Страница 10 успешно обработана
Страница 20 успешно обработана
Страница 30 успешно обработана


In [ ]:
# Avito Parsing